# Import Libraries

In [2]:
import numpy as np 
import pandas as pd
import warnings 
#import nltk
#nltk.download()
import re

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


## Upload Unprocessed CSV file

In [4]:
df = pd.read_csv('Unprocessed Job Data.csv', sep=";", encoding='latin-1', escapechar='\\')

In [6]:
df.head()

,Job_title,Company_name,Location,Salary_fork,Rating,Company_Overview,Job_description,"Avg_base_salary,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
0,Data Engineer Flink,Egrove Systems Corporation,"Dallas, TX",Employer Provided Salary:$51.83 Per Hour,4.4,"Company Overview,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","Java, Bigdata, Azure & Flink,,,,,,,,,,,,,,,,,,...","Average Base Salary Estimate,,,,,,,,,,,,,,,,,,..."
1,Data Engineer,Overstock.com Inc.,Remote,Employer Provided Salary:$87K - $108K,3.1,"Company Overview,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","We Are Overstock:,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","Average Base Salary Estimate,,,,,,,,,,,,,,,,,,..."
2,Data Engineer I,Nucor Business Technology,"Charlotte, NC",$67K - $86K (Glassdoor est.),3.9,"Company Overview,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","Job Details,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","Average Base Salary Estimate,,,,,,,,,,,,,,,,,,..."
3,Data Engineer 1,Kagr Llc,"Foxborough, MA",$70K - $98K (Glassdoor est.),3.4,"Company Overview,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","SUMMARY:,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","Average Base Salary Estimate,,,,,,,,,,,,,,,,,,..."
4,BI Data & Visualization Engineer 1,G&J Pepsi-Cola Bottlers,"Columbus, OH",$65K - $96K (Glassdoor est.),3.6,"Company Overview,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","Key Responsibilities,,,,,,,,,,,,,,,,,,,,,,,,,,...","Average Base Salary Estimate,,,,,,,,,,,,,,,,,,..."


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3766 entries, 0 to 3765
Data columns (total 8 columns):
 #   Column                                                                                                                                                                                                                          Non-Null Count  Dtype 
---  ------                                                                                                                                                                                                                          --------------  ----- 
 0   Job_title                                                                                                                                                                                                                       3766 non-null   object
 1   Company_name                                                                                                                         

### 1. Create a function to generate ids and make a new column named JOB_ID using it.


In [10]:
def generate_job_id(index):
    """Generates a unique job ID in the format JOB0001, JOB0002, etc."""
    return f"JOB{str(index + 1).zfill(4)}"

# Assuming your DataFrame is named 'df'
df['JOB_ID'] = df.index.map(generate_job_id)

In [12]:
df['JOB_ID'].head()

0    JOB0001
1    JOB0002
2    JOB0003
3    JOB0004
4    JOB0005
Name: JOB_ID, dtype: object

### Checking Missing Values

In [14]:
missing_values = df.isnull().sum()
print(missing_values)

Job_title                                                                                                                                                                                                                         0
Company_name                                                                                                                                                                                                                      0
Location                                                                                                                                                                                                                          4
Salary_fork                                                                                                                                                                                                                       0
Rating                                                                                  

### Covert Job_description column data into String 

In [16]:
df['Job_description'] = df['Job_description'].astype(str)

### 2. Create a function to clean the messy text and use this function to make a new column named Clean_Job_Descripton.

In [26]:
# Clean the Job_description column
df['Clean_Job_description'] = df['Job_description'].str.replace('\r\n', '', regex=False).str.strip().str.replace('"', '')
df['Clean_Job_description'] = df['Clean_Job_description'].apply(lambda x: re.sub(r',+', '', str(x)))

In [28]:
df['Clean_Job_description'].head()

0    Java Bigdata Azure & FlinkFlink Sr. Developer ...
1    We Are Overstock:At Overstock.com we believe t...
2    Job DetailsDivision: Nucor Business Technology...
3    SUMMARY:The Data Engineer 1 position will be p...
4    Key ResponsibilitiesLeads development of repor...
Name: Clean_Job_description, dtype: object

### 3. Create a function to filter out skills from the Clean_Job_Descripton column and make a separate column named skills that stores skills in a list format.


In [32]:
def extract_skills(Clean_Job_description):
  """Extracts skills from a job description string and returns them as a list.

  Args:
    job_description: The job description string.

  Returns:
    A list of extracted skills.
  """

  # Define a regular expression pattern to match common skill words
  skill_pattern = r"\b(Python|Java|C++|JavaScript|SQL|HTML|CSS|Machine Learning|Deep Learning|Data Science|Data Analytics|Cloud Computing|AWS|Azure|GCP|DevOps|Agile|Scrum|Project Management|Software Engineering)\b"

  # Find all matches of the pattern in the job description
  skills = re.findall(skill_pattern, Clean_Job_description, re.IGNORECASE)

  return skills

# Apply the extract_skills function to the Job_Description column and create a new Skills column
df["Skills"] = df["Clean_Job_description"].apply(extract_skills)

# Print the DataFrame with the extracted skills
print(df["Skills"])

0           [Java, Azure, Java, Java, Azure, Java, Azure]
1       [SQL, SQL, GCP, Python, Java, C, machine learn...
2       [SQL, Azure, SQL, Azure, DevOps, Agile, Azure,...
3       [SQL, Agile, AWS, Azure, SQL, Agile, AWS, Azur...
4       [Agile, Scrum, Agile, Scrum, SQL, SQL, SQL, Py...
                              ...                        
3761                              [SQL, SQL, SQL, Python]
3762    [Python, SQL, project management, Agile, Pytho...
3763    [data science, data science, data science, Pyt...
3764    [SQL, SQL, SQL, SQL, SQL, Python, SQL, SQL, SQ...
3765    [data science, SQL, Python, data science, SQL,...
Name: Skills, Length: 3766, dtype: object
